<a href="https://colab.research.google.com/github/danangcorp/sql-dqlab/blob/main/DQProject_SQL_Optimasi_DQPizza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><span style='color:#0B2F9F'>Latar Belakang</span></b>

Di sebuah sudut kota Jakarta, telah didirikan sebuah restoran pizza bernama DQPizza. Selama satu tahun terakhir, telah dilakukan pencatatan atas setiap transaksi yang terjadi. Mulai dari pemesanan menu hingga pembayaran pelanggan. Banyak data telah dikumpulkan dan dicatat. Namun, belum dilakukan pemanfaatan optimal terhadap informasi yang tersedia. Pemilik restoran menginginkan agar data ini dapat dianalisis guna menemukan ruang untuk perbaikan dan melakukan efisiensi operasional sehingga diharapkan dapat meningkatkan penjualan pizza, menekan biaya operasional dan meningkatkan customer experience


# <b><span style='color:#0B2F9F'>Set up</span></b>

Dibutuhkan proses autentikasi dari Google Colab ke Google Big Query. Ikuti langkah berikut https://drive.google.com/file/d/1gW8alZ_PrvcrsieqWCHOR4ssLI_25BRc/view untuk detail step-by-step nya

In [ ]:
# Import library yang dibutuhkan
from google.colab import auth, data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq

# Proses autentikasi akun
auth.authenticate_user()
print('Authenticated')

In [ ]:
# Inisialisasi project_id dan dataset_id
project_id = 'nezaproject-123456'
dataset_id = 'dq_pizza'

# Buat BigQuery client
client = bigquery.Client(project = project_id)
dataset_ref = bigquery.Dataset(f'{project_id}.{dataset_id}')
dataset = client.create_dataset(dataset_ref, exists_ok = True)

# <b><span style='color:#0B2F9F'>Data Digunakan</span></b>

In [ ]:
%%bigquery --project {project_id} --verbose

CREATE TABLE IF NOT EXISTS dq_pizza.tbl_all_transaction AS
WITH temp_order AS (
  SELECT
    order_id,
    customer_id,
    order_maker_id,
    PARSE_DATE('%m/%d/%Y', order_date) AS order_date,
    PARSE_TIME('%H:%M:%S', order_time) AS order_time,
    PARSE_TIME('%H:%M:%S', completion_time) AS completion_time,
    is_complain,
    complain_detail
  FROM `dqlab-9876543.dq_pizza.orders`
  WHERE order_id IS NOT NULL
), temp_order_detail AS (
  SELECT
    order_details_id,
    order_id,
    pizza_id,
    quantity
  FROM `dqlab-9876543.dq_pizza.order_details`
  WHERE order_details_id IS NOT NULL
), temp_pizza AS (
  SELECT DISTINCT
    pizza_id,
    pizza_type_id,
    size AS pizza_size,
    CAST(REPLACE(price, 'IDR', '') AS FLOAT64) AS pizza_price,
    CAST(REPLACE(production_cost, 'IDR', '') AS FLOAT64) AS pizza_production_cost
  FROM `dqlab-9876543.dq_pizza.pizzas`
), temp_pizza_type AS (
  SELECT DISTINCT
    pizza_type_id,
    UPPER(name) AS pizza_name,
    UPPER(category) AS pizza_category,
    ingredients AS pizza_ingredients
  FROM `dqlab-9876543.dq_pizza.pizza_types`
), temp_customer AS (
  SELECT DISTINCT
    customer_id,
    customer_name,
    gender AS customer_gender,
    DATE_DIFF(CURRENT_DATE(), CAST(birth_date AS DATE), YEAR) AS customer_age
  FROM `dqlab-9876543.dq_pizza.customers`
)
  SELECT
    o.order_id,
    od.order_details_id,
    od.pizza_id,
    o.customer_id,
    o.order_maker_id,
    o.order_date,
    c.customer_name,
    c.customer_gender,
    c.customer_age,
    pt.pizza_name,
    pt.pizza_category,
    p.pizza_size,
    p.pizza_price,
    p.pizza_production_cost,
    od.quantity,
    pt.pizza_ingredients,
    o.order_time,
    o.completion_time,
    o.is_complain,
    o.complain_detail,
    CURRENT_TIMESTAMP() AS created_date
  FROM temp_order AS o
  INNER JOIN temp_order_detail AS od ON o.order_id = od.order_id
  LEFT JOIN temp_pizza AS p ON od.pizza_id = p.pizza_id
  LEFT JOIN temp_pizza_type pt ON p.pizza_type_id = pt.pizza_type_id
  LEFT JOIN temp_customer c ON o.customer_id = c.customer_id

## **Area 1 - Performance Summary**

_Bagaimana tren jumlah transaksi dan total pendapatan penjualan pizza di DQPizza setiap bulan selama periode 2024? Adakah pola menarik yang kamu temukan?_

**Hint :**
* Tentukan periode waktu yang ingin dianalisis, yaitu setiap bulan dalam satu tahun.
* Kelompokkan data transaksi berdasarkan bulan terjadinya pembelian.
* Hitung jumlah transaksi yang terjadi di setiap bulan untuk melihat seberapa ramai penjualan pada periode tersebut.
* Hitung juga total pendapatan di setiap bulan untuk mengetahui seberapa besar nilai penjualan yang dihasilkan.
* Urutkan hasilnya dari bulan awal hingga akhir tahun agar terlihat jelas tren pergerakan penjualan dari waktu ke waktu.
* Analisis hasilnya untuk menemukan pola seperti bulan dengan penjualan tertinggi, periode sepi transaksi, atau potensi momen promosi.



In [ ]:
%%bigquery summary_trx_per_month --project {project_id} --verbose

SELECT
  EXTRACT(MONTH FROM order_date) AS month_number,
  FORMAT_DATE('%B', order_date) AS month_name,
  COUNT(DISTINCT order_id) AS total_transactions,
  SUM(pizza_price * quantity) AS total_revenue
FROM `dq_pizza.tbl_all_transaction`
WHERE EXTRACT(YEAR FROM order_date) = 2024
GROUP BY month_number, month_name
ORDER BY month_number;


In [ ]:
# Tampilkan hasilnya
display(summary_trx_per_month)

In [ ]:
#@title ***DQPizza Performance Summary in 2024*** {display-mode: 'form'}

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows = 2,
    cols = 1,
    shared_xaxes = True,
    row_heights = [0.50, 0.75],
    vertical_spacing = 0.01
)

# Line chart pada subplot 1
fig.add_trace(
    px.line(
        summary_trx_per_month,
        x = 'month_name',
        y = 'total_trx',
        markers = True,
        color_discrete_sequence=['#03c03c'],
        labels = dict(
            x = 'month_name',
            y = 'total_trx'
        ),
        line_shape='linear'
    ).data[0],
    row = 1,
    col = 1
)

for i in [0, 11]:
    last_month_trx = summary_trx_per_month.loc[i, 'month_name']
    last_value_trx = summary_trx_per_month.loc[i, 'total_trx']
    if(i == 11):
        add_y = -100
    else:
        add_y = 100
    fig.add_annotation(
        x = last_month_trx,
        y = last_value_trx + add_y,
        text = f'<b>{last_month_trx}</b><br>Total Transaction : {last_value_trx}',
        showarrow = False,
        font = dict(
            color='#03c03c',
            family = "sans serif",
            size = 15,
        )
    )

fig.update_layout(
    yaxis = dict(
        showline = False,
        showgrid = False,
        showticklabels = False,
    )
)

# Add annotation for the maximum value with distance
max_month_trx = summary_trx_per_month.loc[summary_trx_per_month['total_trx'].idxmax(), 'month_name']
max_value_trx = summary_trx_per_month['total_trx'].max()

fig.add_annotation(
    x = max_month_trx,
    y = max_value_trx + 100,
    text = f'<b>{max_month_trx}</b><br>Best Transaction : {max_value_trx}',
    showarrow = False,
    font = dict(
        color='#02862a',
        family = "sans serif",
        size = 15,
    )
)

# Add line chart to the second row
fig.add_trace(
    px.bar(
        summary_trx_per_month,
        x = 'month_name',
        y = 'total_revenue',
        color_discrete_sequence=['#03c03c'],
        labels = dict(
            x = 'month_name',
            y = 'total_revenue'
          ),
          text = ['IDR ' + str(round(value / 1000000, 1)) + 'jt' for value in summary_trx_per_month['total_revenue']]
        ).data[0],
        row = 2,
        col = 1
    )

    # Update layout
fig.update_layout(
    height = 600,
    width = 1200,
    bargap = 0.05,
    showlegend = False,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    paper_bgcolor = 'rgba(0, 0, 0, 0)',
    title = {
        'text': '<b>DQPizza Performance Summary in 2024</b>',
        'font': {'color': '#02862a', 'size': 20},
        'x' : 0.085
    }
)

"""
    fig.update_traces(
        marker = dict(
            color= ['#02862a' if val == 1 else '#03c03c' for val in summary_trx_per_month['rank_revenue']]
        )
    )
"""

fig.update_xaxes(showgrid=False)
fig.update_yaxes(
    showgrid  = False,
    visible = False,
    tickfont_color = "white"
)

fig.show()

**Kesimpulan :**     

Maka dapat disimpulkan total pendapatan Tertinggi DQPizza terjadi pada bulan juli dengan total pendapatan IDR 587,7 jt dan total transaksi 1935

## **Area 2 - Product**

_Jenis pizza apa saja yang paling banyak dipesan berdasarkan data transaksi DQPizza? Tampilkan top 10 pizza_

* Gunakan data transaksi penjualan pizza dari dataset DQPizza.
* Gabungkan informasi kategori dan nama pizza agar setiap jenis produk dapat dikenali dengan jelas.
* Hitung jumlah pesanan untuk setiap jenis pizza guna melihat tingkat popularitasnya.
* Urutkan hasilnya dari yang paling sering dipesan hingga paling jarang.
* Tampilkan hanya 10 jenis pizza teratas agar fokus pada menu favorit pelanggan.
* Gunakan hasil ini untuk menganalisis preferensi pelanggan dan menentukan strategi promosi atau pengelolaan stok menu.

In [ ]:
%%bigquery total_order_per_pizzaname --project {project_id} --verbose

SELECT
  -- 2. Menggabungkan kategori dan nama pizza dengan pemisah '|'
  CONCAT(pizza_category, ' | ', pizza_name) AS pizza_name,

  -- 3. Menghitung jumlah TRANSAKSI / ORDER UNIK
  COUNT(DISTINCT order_id) AS total_order
FROM
  -- 1. Menggunakan data transaksi penjualan pizza
  dq_pizza.tbl_all_transaction
GROUP BY
  -- Mengelompokkan berdasarkan nama dan kategori
  pizza_category,
  pizza_name
ORDER BY
  -- 4. Mengurutkan hasil dari yang paling banyak dipesan
  total_order DESC
LIMIT 10; -- 5. Menampilkan hanya 10 jenis pizza teratas


In [ ]:
# Tampilkan hasilnya
display(total_order_per_pizzaname)

In [ ]:
#@title ***Total Order per Pizza Name*** {display-mode: 'form'}

import plotly.express as px

total_order_per_pizzaname.sort_values(
    by = 'total_order',
    ascending = True,
    ignore_index = True,
    inplace = True
)

total_order_per_pizzaname['pizza_name'] = total_order_per_pizzaname['pizza_name'].str.title()
total_order_per_pizzaname['pizza_name'] = '<b>' + total_order_per_pizzaname['pizza_name'].str.replace(' |', '</b> |')
total_order_per_pizzaname['Color'] = total_order_per_pizzaname['total_order'].apply(
    lambda x: 'Top' if x == total_order_per_pizzaname['total_order'].max() else 'Other'
)

fig = px.bar(
    total_order_per_pizzaname,
    x = 'total_order',
    y = 'pizza_name',
    orientation = 'h',
    color = 'Color',
    color_discrete_map = {
        'Other': '#85ff7a',
        'Top': '#2db83d'
    },
    text_auto = True
)

fig.update_layout(
    width = 950,
    height = 450,
    xaxis_title = '',
    yaxis_title = '',
    showlegend = False,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = '<b><i>Best Seller</i> Pizza Berdasarkan Nama</b><br><sup><sup>Periode 2024</sup></sup>',
        font = dict(color = '#D19C4B')
    )
)

fig.update_xaxes(showticklabels = False)

fig.update_traces(
    textposition = 'inside',
    hovertemplate = '<b>Pizza %{label}</b><br>Total Order = %{value}'
)

fig.show()


**Kesimpulan :**

Berdasarkan analisis "Total Order per Pizza Name", dapat disimpulkan bahwa jenis pizza kategori "CLASSIC" mendominasi daftar 10 pizza teratas yang paling banyak dipesan. Tiga pizza terlaris adalah "THE CLASSIC DELUXE PIZZA", "THE HAWAIIAN PIZZA", dan "THE PEPPERONI PIZZA". Kategori "CHICKEN" juga cukup populer dengan empat jenis pizza masuk dalam daftar 10 teratas. Hal ini menunjukkan bahwa pelanggan DQPizza memiliki preferensi yang kuat terhadap pizza dengan topping klasik dan ayam.

## **Area 3 - Service**

_Pada jam berapa pembuatan pizza di DQPizza paling banyak dilakukan di setiap hari dalam seminggu berdasarkan data?_

* Gunakan data transaksi dari dataset DQPizza yang memuat waktu pemesanan dan jumlah pizza yang dibuat.
* Identifikasi hari dan jam dari setiap transaksi untuk mengetahui kapan aktivitas pembuatan pizza terjadi.
* Hitung total jumlah pizza yang dibuat pada setiap kombinasi hari dan jam.
* Hitung rata-rata jumlah pizza yang dibuat pada setiap jam di setiap hari untuk menemukan pola konsistensi aktivitas produksi.
* Urutkan hasilnya berdasarkan rata-rata jumlah tertinggi agar terlihat waktu dengan aktivitas produksi paling sibuk.
* Gunakan hasil analisis ini untuk mengoptimalkan penjadwalan karyawan dan perencanaan bahan baku di dapur.

In [ ]:
%%bigquery num_qty_per_hour --project {project_id} --verbose

WITH
DailyHourlySum AS (
  SELECT
    order_date,
    EXTRACT(HOUR FROM order_time) AS jam,
    SUM(quantity) AS total_pizza_harian_per_jam
  FROM
    `dq_pizza.tbl_all_transaction`
  GROUP BY
    1, 2
),
AverageHourlyStats AS (
  SELECT
    FORMAT_DATE('%A', order_date) AS nama_hari,
    jam,
    AVG(total_pizza_harian_per_jam) AS rata_rata_pizza_per_jam_slot
  FROM
    DailyHourlySum
  GROUP BY
    1, 2
)
SELECT
  nama_hari AS day_trx,
  jam AS hour_trx,
  ROUND(rata_rata_pizza_per_jam_slot, 2) AS avg_quantity
FROM
  AverageHourlyStats
ORDER BY
  avg_quantity DESC;


In [ ]:
display(num_qty_per_hour)

In [ ]:
#@title ***Rata - Rata Pizza Dibuat Tiap Jam Setiap Harinya*** {display-mode: 'form'}

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

avg_qty_per_hour = num_qty_per_hour.groupby(['hour_trx'], as_index = False).agg(avg_qty_per_hour = ('avg_quantity', 'mean'))

day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
num_qty_per_hour['day_trx'] = pd.Categorical(num_qty_per_hour['day_trx'], categories=day_order, ordered=True)

num_qty_per_hour_pivot = num_qty_per_hour.pivot(
    index = 'day_trx',
    columns = 'hour_trx',
    values = 'avg_quantity'
).sort_values(by = 'day_trx', ascending = False, ignore_index = False)

fig = make_subplots(
    rows = 2,
    cols = 1,
    shared_xaxes = True,
    row_heights = [0.25, 0.75],
    vertical_spacing = 0.01
)

fig.add_trace(
    px.bar(
        avg_qty_per_hour,
        x = 'hour_trx',
        y = 'avg_qty_per_hour',
        labels = dict(
            x = 'month_trx',
            y = 'total_revenue'
        ),
        color = 'avg_qty_per_hour',
    ).data[0],
    row = 1,
    col = 1
)

fig.update_layout(
    yaxis = dict(
        showline = False,
        showgrid = False,
        showticklabels = False,
    )
)

fig.add_trace(
    px.imshow(
        num_qty_per_hour_pivot
    ).data[0],
    row = 2,
    col = 1
)

fig.update(
    layout_coloraxis_showscale = False
)

fig.update_xaxes(
    automargin = True
)

fig.update_layout(
    height = 600,
    width = 700,
    bargap = 0.05,
    coloraxis_colorscale = 'oranges',
    xaxis_tickangle = 0,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    paper_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = '<b>Rata - Rata Pizza Dibuat Tiap Jam Setiap Harinya</b><br><sup><sup>Periode 2024</sup></sup>',
        font = dict(
            color = '#B54B1F'
        )
    )
)

fig.show()

**Kesimpulan :**

Berdasarkan analisis rata-rata jumlah pizza yang dibuat setiap jam setiap harinya, dapat disimpulkan bahwa waktu tersibuk untuk pembuatan pizza di DQPizza adalah di sekitar jam makan siang, terutama pada hari Sabtu pukul 12:00 dengan rata-rata 24.46 pizza dibuat per jam. Hari dan jam lain yang menunjukkan aktivitas tinggi termasuk Selasa pukul 12:00 dan 13:00, serta Minggu dan Rabu pukul 12:00. Pola ini mengindikasikan bahwa DQPizza mengalami lonjakan pesanan pada jam-jam makan siang di pertengahan hingga akhir minggu. Informasi ini penting untuk mengoptimalkan penjadwalan staf dapur dan manajemen inventaris bahan baku.

## **Area 4 - Proportion Complain**
_Bagaimana perbandingan jumlah pesanan yang disertai keluhan (complain) dan yang tidak disertai keluhan berdasarkan data?_

* Gunakan data transaksi dari dataset DQPizza yang mencatat apakah suatu pesanan memiliki keluhan atau tidak.
* Kelompokkan data berdasarkan status keluhan untuk membedakan antara pesanan “Complain” dan “No Complain”.
* Hitung jumlah pesanan pada masing-masing kelompok untuk mengetahui proporsi antara kedua jenis transaksi tersebut.
* Gunakan hasilnya untuk memahami tingkat kepuasan pelanggan secara umum.
* Analisis lebih lanjut dapat dilakukan untuk mencari tahu penyebab utama keluhan atau bagian proses pelayanan yang perlu ditingkatkan.

In [ ]:
%%bigquery proportion_complain --project {project_id} --verbose

SELECT
  CASE
    WHEN is_complain = 1 THEN 'Complain'
    ELSE 'No Complain'
  END AS is_complain_label,
  COUNT(*) AS total_complain
FROM
  dq_pizza.tbl_all_transaction
GROUP BY
  is_complain_label;

In [ ]:
display(proportion_complain)

In [ ]:
#@title ***Proporsi Kejadian Komplain di DQPizza*** {display-mode: 'form'}

# Import library untuk visualisasi
import plotly.express as px

# Hitung total data
total_data = proportion_complain['total_complain'].sum()

# Warna
hijau_pucat = '#E0ECE4'
merah = '#FF4B5C'

# Buat pie chart
fig = px.pie(
    values = proportion_complain['total_complain'],
    names = proportion_complain['is_complain_label'],
    color_discrete_sequence = [hijau_pucat, merah],
    hole = 0.65
)

# Atur posisi label
fig.update_traces(
    textposition = 'outside',
    textinfo = 'percent+label',
    hovertemplate='<b>%{label}</b><br>%{value} Customers'
)

# Atur luas grafik, hapus legend dan beri judul
fig.update_layout(
    width = 800,
    height = 600,
    showlegend = False,
    margin = dict(l=160, r=200, t=100, b=30),
    title = dict(
        text = f"<b>Proporsi Kejadian Komplain di DQPizza</b><br>",
        font = dict(
            size = 25,
            color = '#757882'
        ),
        y = 0.92,
        x = 0.46
    )
)

# Berikan informasi total pelanggan di tengah donut chart
fig.add_annotation(
    text = f'Total Transaksi<br><b><span style="font-size: 28px;">{total_data}</b></span>',
    x = 0.5,
    y = 0.5,
    showarrow = False,
    font = dict(size = 30)
)

# Tampilkan grafik
fig.show()

**Kesimpulan :**

Berdasarkan analisis proporsi kejadian komplain di DQPizza, dapat disimpulkan bahwa mayoritas transaksi (43,746) tidak disertai keluhan, sementara sebagian kecil transaksi (4,873) menghasilkan komplain. Proporsi ini menunjukkan bahwa DQPizza secara umum berhasil memberikan pelayanan yang memuaskan bagi sebagian besar pelanggannya. Namun, jumlah komplain yang ada tetap perlu diperhatikan untuk mengidentifikasi akar masalah dan melakukan perbaikan berkelanjutan demi meningkatkan kepuasan pelanggan secara keseluruhan.

## **Area 5 - Total Detail Complain**
_Jenis keluhan apa saja yang paling sering muncul pada pesanan pelanggan berdasarkan data transaksi?_

* Gunakan data transaksi dari dataset DQPizza yang memuat informasi detail keluhan pelanggan.
* Saring data agar hanya mencakup pesanan yang memiliki keluhan.
* Kelompokkan data berdasarkan jenis atau kategori keluhan yang tercatat.
* Hitung jumlah kemunculan setiap jenis keluhan untuk mengetahui keluhan yang paling sering terjadi.
* Urutkan hasilnya dari jumlah terkecil hingga terbesar agar terlihat pola umum dan prioritas perbaikan layanan.
* Gunakan hasil ini untuk mengidentifikasi area yang perlu ditingkatkan, seperti kualitas produk, waktu pengantaran, atau pelayanan pelanggan.

In [ ]:
%%bigquery total_detail_complain --project {project_id} --verbose

SELECT
  complain_detail,
  COUNT(*) AS jumlah_complain
FROM
  dq_pizza.tbl_all_transaction
WHERE
  is_complain = 1 AND complain_detail IS NOT NULL
GROUP BY
  complain_detail
ORDER BY
  jumlah_complain ASC;

In [ ]:
display(total_detail_complain)

In [ ]:
#@title ***Detail Komplain yang Terjadi di DQPizza*** {display-mode: 'form'}

import plotly.express as px

total_detail_complain['Color'] = total_detail_complain['jumlah_complain'].apply(
    lambda x: 'Top' if x == total_detail_complain['jumlah_complain'].max() else 'Other'
)

fig = px.bar(
    total_detail_complain,
    x = 'jumlah_complain',
    y = 'complain_detail',
    orientation = 'h',
    color = 'Color',
    color_discrete_map = {
        'Other': '#ffb3b3',
        'Top': '#ff0000'
    },
    text_auto = True
)

fig.update_layout(
    width = 950,
    height = 450,
    xaxis_title = '',
    yaxis_title = '',
    showlegend = False,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = '<b>Detail Komplain yang Terjadi di DQPizza</b><br><sup><sup>Periode 2024</sup></sup>',
        font = dict(color = '#ff0000')
    )
)

fig.update_xaxes(showticklabels = False)

fig.update_traces(
    textposition = 'inside',
    hovertemplate = '<b>Pizza %{label}</b><br>Total Order = %{value}'
)

fig.show()


**Kesimpulan :**

Berdasarkan analisis detail keluhan yang paling sering muncul, dapat disimpulkan bahwa keluhan "Missing toppings" merupakan yang paling umum terjadi dengan jumlah 1186. Diikuti oleh "Pizza was burnt" dengan 1030 keluhan, dan "Order took too long" dengan 693 keluhan. Keluhan lain seperti "Received wrong pizza", "Wrong pizza size", "Drink was missing", "Dough undercooked", dan "Rude staff at counter" terjadi dengan frekuensi yang lebih rendah. Informasi ini menunjukkan bahwa DQPizza perlu memprioritaskan perbaikan dalam memastikan kelengkapan topping, kualitas pemanggangan pizza, dan efisiensi waktu pemesanan untuk meningkatkan kepuasan pelanggan.

## **Area 6 - Total Complain Per Order Maker**
_Bagaimana distribusi dan jenis keluhan pelanggan yang diterima oleh masing-masing pembuat pizza (order maker) berdasarkan data transaksi?_

* Gunakan data transaksi dari dataset DQPizza yang memuat informasi tentang pembuat pizza (order maker) dan detail keluhan pelanggan.
* Identifikasi setiap keluhan yang tercatat untuk setiap pembuat pizza.
* Hitung jumlah kemunculan setiap jenis keluhan pada masing-masing pembuat pizza untuk mengetahui pola atau tren tertentu.
* Ubah tampilan data agar setiap jenis keluhan menjadi kolom, sehingga mudah dibandingkan antar pembuat pizza.
* Urutkan hasilnya berdasarkan pembuat pizza untuk melihat siapa yang paling sering menerima keluhan dan jenis keluhannya.
* Gunakan hasil analisis ini untuk menilai kinerja masing-masing pembuat pizza, serta menentukan area pelatihan atau peningkatan kualitas produksi yang dibutuhkan.

In [ ]:
%%bigquery total_complain_per_ordermaker --project {project_id} --verbose

SELECT
  order_maker_id,
  SUM(CASE WHEN complain_detail = 'Dough undercooked' THEN 1 ELSE 0 END) AS Dough_undercooked,
  SUM(CASE WHEN complain_detail = 'Drink was missing' THEN 1 ELSE 0 END) AS Drink_was_missing,
  SUM(CASE WHEN complain_detail = 'Missing toppings' THEN 1 ELSE 0 END) AS Missing_toppings,
  SUM(CASE WHEN complain_detail = 'Order took too long' THEN 1 ELSE 0 END) AS Order_took_too_long,
  SUM(CASE WHEN complain_detail = 'Pizza was burnt' THEN 1 ELSE 0 END) AS Pizza_was_burnt,
  SUM(CASE WHEN complain_detail = 'Received wrong pizza' THEN 1 ELSE 0 END) AS Received_wrong_pizza,
  SUM(CASE WHEN complain_detail = 'Rude staff at counter' THEN 1 ELSE 0 END) AS Rude_staff_at_counter,
  SUM(CASE WHEN complain_detail = 'Wrong pizza size' THEN 1 ELSE 0 END) AS Wrong_pizza_size
FROM
  dq_pizza.tbl_all_transaction
WHERE
  is_complain = 1
GROUP BY
  order_maker_id
ORDER BY
  order_maker_id;

In [ ]:
display(total_complain_per_ordermaker)

In [ ]:
#@title ***Jumlah Detail Komplain per Pizza Order Maker*** {display-mode: 'form'}

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = px.imshow(
    total_complain_per_ordermaker.set_index('order_maker_id'),
    text_auto = True,
)

fig.update(
    layout_coloraxis_showscale = False
)

fig.update_layout(
    height = 1000,
    width = 800,
    coloraxis_colorscale = 'reds',
    xaxis_tickangle = 0,
    plot_bgcolor = 'rgba(0, 0, 0, 0)',
    paper_bgcolor = 'rgba(0, 0, 0, 0)',
    title = dict(
        text = '<b>Jumlah Detail Komplain per <i>Pizza Order Maker</i></b><br><sup><sup>Periode 2024</sup></sup>',
        font = dict(
            color = '#B54B1F',
            size = 20
        )
    ),
    margin=dict(t=150)
)

fig.update_xaxes(
    tickangle = -15,
    side = 'top',
    automargin = True,
    title = None,
    tickfont = dict(size=11)
)

fig.update_yaxes(
    title = None,
    tickfont = dict(size=11)
)

fig.show()

**Kesimpulan :**

Berdasarkan analisis jumlah detail komplain per pizza order maker, dapat disimpulkan bahwa setiap pembuat pesanan memiliki distribusi keluhan yang berbeda-beda. Beberapa pembuat pesanan menunjukkan kecenderungan lebih tinggi pada jenis keluhan tertentu. Contohnya, Order Maker EMP000756 memiliki jumlah komplain "Pizza was burnt" yang signifikan (140), sementara Order Maker EMP000437 menerima komplain "Missing toppings" paling banyak (155). Pola ini menunjukkan perlunya pelatihan atau penyesuaian proses yang spesifik untuk setiap order maker guna mengatasi akar masalah dari jenis keluhan yang paling sering mereka terima.

## **Area 7 - Customer**

_Bagaimana distribusi usia pelanggan berdasarkan kategori gender?_

* Gunakan data pelanggan dari dataset DQPizza yang memuat informasi usia dan jenis kelamin pelanggan.
* Kelompokkan data berdasarkan kategori gender, seperti laki-laki dan perempuan.
* Amati sebaran usia di masing-masing kelompok untuk melihat apakah terdapat perbedaan karakteristik pelanggan berdasarkan gender.
* Analisis pola umum, misalnya kelompok usia mana yang paling dominan dalam tiap gender.
* Gunakan hasil analisis ini untuk mendukung strategi pemasaran yang lebih tepat sasaran, seperti segmentasi promosi atau penawaran menu khusus bagi kelompok pelanggan tertentu

In [ ]:
%%bigquery age_and_gender --project {project_id} --verbose

SELECT
  customer_gender,
  customer_age
FROM
  `dq_pizza.tbl_all_transaction`
WHERE customer_gender IS NOT NULL AND customer_age IS NOT NULL

In [ ]:
display(age_and_gender)

In [ ]:
# @title ***Distribusi Usia Pelanggan*** {display-mode: 'form'}

# Import library yang dibutuhkan
import plotly.figure_factory as ff

def distribution_plot(data_cat_1, data_cat_2, label, column_name):
    # Group data together
    data_cat_1 = data_cat_1[column_name]
    data_cat_2 = data_cat_2[column_name]
    hist_data = [data_cat_1, data_cat_2]
    group_labels = label

    # Create distplot with custom bin_size
    fig = ff.create_distplot(
        hist_data,
        group_labels,
        show_hist = False,
        show_rug = False
    )

    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        title = dict(
            text = f"<b>Distribusi Usia Pelanggan</b>",
            font = dict(
                size = 28,
                color = 'black'
            ),
            y = 0.92,
            x = 0.5
        )
    )

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')

    # Tampilkan visualisasi
    fig.show()

male_cust = age_and_gender[age_and_gender['customer_gender'] == 'M']
female_cust = age_and_gender[age_and_gender['customer_gender'] == 'F']
label_grup =  ['Male', 'Female']

distribution_plot(male_cust, female_cust, label_grup, 'customer_age')

**Kesimpulan :**

Berdasarkan analisis distribusi usia pelanggan berdasarkan gender, dapat disimpulkan bahwa DQPizza memiliki pelanggan dari berbagai rentang usia, dari sekitar {int(age_and_gender['customer_age'].min())} hingga {int(age_and_gender['customer_age'].max())} tahun. Pelanggan **laki-laki sedikit lebih dominan** dibandingkan pelanggan perempuan. Visualisasi distribusi usia menunjukkan pola yang serupa untuk kedua gender, dengan konsentrasi terbesar pada kelompok usia tertentu. Informasi ini penting untuk menargetkan strategi pemasaran secara efektif.

## **Area 8 - Preferensi Kategori Pizza per Order**
_Bagaimana preferensi kategori pizza berdasarkan gender pelanggan pada data transaksi?_

* Gunakan data transaksi dari dataset DQPizza yang mencakup informasi tentang jenis kelamin pelanggan dan kategori pizza yang dipesan.
* Kelompokkan data berdasarkan kombinasi antara gender pelanggan dan kategori pizza.
* Hitung jumlah pesanan di setiap kategori untuk masing-masing gender agar terlihat kategori pizza mana yang paling disukai oleh setiap kelompok pelanggan.
* Urutkan hasilnya berdasarkan gender dan jumlah pesanan terbanyak untuk memperjelas perbandingan antar kelompok.
* Gunakan hasil analisis ini untuk merancang strategi pemasaran yang lebih terarah, seperti promosi menu tertentu untuk segmen pelanggan laki-laki atau perempuan.

In [ ]:
%%bigquery pizza_category_by_gender --project {project_id} --verbose

SELECT
  customer_gender,
  pizza_category,
  COUNT(*) AS total_pizza_category
FROM
  dq_pizza.tbl_all_transaction
WHERE customer_gender IS NOT NULL AND pizza_category IS NOT NULL
GROUP BY
  customer_gender,
  pizza_category
ORDER BY
  customer_gender,
  total_pizza_category DESC;

In [ ]:
display(pizza_category_by_gender)

In [ ]:
#@title ***Preferensi Kategori Pizza per Order*** {display-mode: 'form'}

import pandas as pd
import numpy as np
import plotly.express as px

pivot = df.pivot_table(
    index='pizza_category',
    columns='customer_gender',
    values='total_pizza_category',
    aggfunc='sum',
    fill_value=0
).reset_index()

gender_cols = [c for c in pivot.columns if c != 'pizza_category']
if len(gender_cols) < 2:
    raise ValueError("Butuh minimal dua kategori gender untuk butterfly chart.")
g1, g2 = gender_cols[0], gender_cols[1]

pivot[f'{g1}_neg'] = -pivot[g1]

fig = px.bar(
    pivot,
    y='pizza_category',
    x=[f'{g1}_neg', g2],
    orientation='h',
    text_auto=True,
    title=f'<b>Preferensi Kategori Pizza per Gender: {g1} vs {g2}</b>',
    labels={'pizza_category': 'Pizza Category'}
)

max_val = int(max(pivot[g1].max(), pivot[g2].max()))
xticks = np.linspace(-max_val, max_val, 9)
xticktext = [str(abs(int(x))) for x in xticks]

fig.update_layout(
    yaxis_title=None,
    barmode='relative',
    bargap=0.2,
    height=700,
    width=1000,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(
        tickmode='array',
        tickvals=xticks,
        ticktext=xticktext,
        title='Jumlah Pesanan'
    ),
    legend_title_text='Gender'
)

fig.update_xaxes(showticklabels=False)
fig.update_traces(texttemplate='%{text}', textposition='inside')

# --- Ubah nama trace agar legend tampil rapi (tanpa _neg) ---
fig.data[0].name = g1
fig.data[1].name = g2

# --- Pastikan nilai teks selalu positif ---
for trace in fig.data:
    vals = np.abs(trace.x)
    trace.text = vals
    trace.textfont = dict(color='white', size=11)

fig.add_vline(x=0, line_width=1, line_color='black')

fig.show()


**Kesimpulan :**
Secara keseluruhan, data menunjukkan bahwa Classic adalah kategori pizza yang paling populer secara mutlak, diminati oleh pelanggan laki-laki (M) dan perempuan (F). Terdapat tren yang jelas di mana pelanggan laki-laki memesan dalam jumlah yang lebih tinggi di semua kategori dibandingkan pelanggan perempuan. Kategori Supreme dan Veggie berada di posisi menengah, sementara Chicken adalah kategori yang paling tidak diminati oleh kedua gender.


Data Source : <i>https://mavenanalytics.io/challenges/maven-pizza-challenge</i> (dengan modifikasi)


---

<br>
<a href="https://www.linkedin.com/in/danang-arya-saputra/"><img src="https://img.shields.io/badge/-© 2025 Danang Arya Saputra-417DAC?style=for-the-badge&logoColor=white"/></a>

<a href="https://dqlab.id/"><img src="https://dqlab.id/files/dqlab/cache/87e30118ebba5ec7d96f6ea8c9dcc10b_x_118_X_55.png" align="left" /></a>
